<a href="https://colab.research.google.com/github/dougyd92/ML-Foudations/blob/main/Notebooks/2_Linear_Regression_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Linear Regression: Interactive Tutorial




In this notebook, we'll implement linear regression three different ways:
1. **scikit-learn** — the standard, production-ready approach
2. **NumPy** — manual least squares to see the math in action
3. **PyTorch** — gradient descent, the foundation for deep learning

Along the way, you'll build intuition for what "fitting a line" really means and why these different approaches all arrive at the same answer.


# Section 0: Setup and Data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

## Our Dataset

We'll use the **California Housing** dataset from scikit-learn. It contains information about housing districts in California, and the target is the **median house value** (in $100,000s).

This is a real dataset — messy, imperfect, and much more interesting than toy examples.

In [ ]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
print(housing.DESCR[:1500])

In [ ]:
X = pd.DataFrame(housing.data, columns=housing.feature_names)
y = pd.Series(housing.target, name='MedHouseVal')

print(f"Dataset shape: {X.shape}")
print(f"Number of samples: {X.shape[0]}")
print(f"Number of features: {X.shape[1]}")
X.head()

In [ ]:
X.describe()

In [ ]:
y.describe()

## Visualizing the Data

Let's look at how a couple of individual features relate to the target.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 4))

axes[0].scatter(X['MedInc'], y, alpha=0.1, s=5)
axes[0].set_xlabel('Median Income')
axes[0].set_ylabel('Median House Value ($100k)')
axes[0].set_title('Income vs House Value')

axes[1].scatter(X['AveRooms'], y, alpha=0.1, s=5)
axes[1].set_xlabel('Average Rooms')
axes[1].set_ylabel('Median House Value ($100k)')
axes[1].set_title('Rooms vs House Value')
axes[1].set_xlim(0, 15)  # Zoom in — there are outliers

axes[2].scatter(X['HouseAge'], y, alpha=0.1, s=5)
axes[2].set_xlabel('House Age (years)')
axes[2].set_ylabel('Median House Value ($100k)')
axes[2].set_title('Age vs House Value')

plt.tight_layout()
plt.show()

Notice how **Median Income** has a pretty clear positive relationship with house value. **Average Rooms** has a weaker trend with lots of spread. **House Age** doesn't show much of a linear pattern at all. This is typical of real data — some features are more useful than others.

## Train-Test Split

Before we do any modeling, we split the data. The model learns from the **training set** and we evaluate on the **test set** to see how it generalizes to data it has never seen.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set:     {X_test.shape[0]} samples")

---
# Section 1: Linear Regression with scikit-learn

This is the standard way you'll do linear regression in practice. scikit-learn handles all the math for you and provides a clean, consistent API.

## Simple Linear Regression (One Feature)

Let's start with just **Median Income** as our single feature to keep things easy to visualize.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# Use only Median Income as the feature
X_train_simple = X_train[['MedInc']]
X_test_simple = X_test[['MedInc']]

# Create and fit the model
model_simple = LinearRegression()
model_simple.fit(X_train_simple, y_train)

# The learned parameters
print(f"Slope (w1):     {model_simple.coef_[0]:.4f}")
print(f"Intercept (w0): {model_simple.intercept_:.4f}")
print(f"\nModel: y = {model_simple.intercept_:.4f} + {model_simple.coef_[0]:.4f} * MedInc")

The slope tells us: for each unit increase in median income (roughly $10,000), the predicted median house value increases by about $41,000. That makes intuitive sense — wealthier neighborhoods have more expensive houses.

In [ ]:
# Predictions on the test set
y_pred_simple = model_simple.predict(X_test_simple)

# Evaluate
mse_simple = mean_squared_error(y_test, y_pred_simple)
r2_simple = r2_score(y_test, y_pred_simple)

print(f"Mean Squared Error: {mse_simple:.4f}")
print(f"R² Score:           {r2_simple:.4f}")

**R²** tells us what fraction of the variance in house values is explained by our model. An R² of ~0.47 means median income alone explains about 47% of the variation — not bad for a single feature, but there's clearly more to the story.

In [ ]:
# Visualize the fit
plt.figure(figsize=(10, 6))
plt.scatter(X_test_simple, y_test, alpha=0.15, s=8, label='Actual')

# Plot the regression line
x_line = np.linspace(X_test_simple.min().values[0], X_test_simple.max().values[0], 100).reshape(-1, 1)
y_line = model_simple.predict(x_line)
plt.plot(x_line, y_line, color='red', linewidth=2, label='Fitted Line')

plt.xlabel('Median Income')
plt.ylabel('Median House Value ($100k)')
plt.title(f'Simple Linear Regression (R² = {r2_simple:.3f})')
plt.legend()
plt.show()

## Multiple Linear Regression (All Features)

Now let's use **all 8 features**. The equation becomes:

ŷ = w₀ + w₁·MedInc + w₂·HouseAge + w₃·AveRooms + ... + w₈·Longitude

In [ ]:
# Fit on all features
model_multi = LinearRegression()
model_multi.fit(X_train, y_train)


In [ ]:
# Predictions
y_pred_multi = model_multi.predict(X_test)
y_pred_multi


In [ ]:
# Evaluate
mse_multi = mean_squared_error(y_test, y_pred_multi)
r2_multi = r2_score(y_test, y_pred_multi)

print(f"Mean Squared Error: {mse_multi:.4f}")
print(f"R² Score:           {r2_multi:.4f}")
print(f"\nImprovement over simple model: R² went from {r2_simple:.3f} to {r2_multi:.3f}")

In [ ]:
# Look at all the coefficients
coef_df = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': model_multi.coef_
}).sort_values('Coefficient', key=abs, ascending=False)

print(f"Intercept: {model_multi.intercept_:.4f}\n")
print(coef_df.to_string(index=False))

In [ ]:
# Visualize: predicted vs actual
plt.figure(figsize=(8, 8))
plt.scatter(y_test, y_pred_multi, alpha=0.15, s=8)
plt.plot([0, 5], [0, 5], 'r--', linewidth=2, label='Perfect predictions')
plt.xlabel('Actual House Value ($100k)')
plt.ylabel('Predicted House Value ($100k)')
plt.title(f'Multiple Linear Regression: Predicted vs Actual (R² = {r2_multi:.3f})')
plt.legend()
plt.xlim(0, 5.2)
plt.ylim(0, 5.2)
plt.gca().set_aspect('equal')
plt.show()

If the model were perfect, all points would fall exactly on the red dashed line. Points above the line are underpredictions; points below are overpredictions. Notice the cluster of points capped at 5.0 — that's the dataset's maximum value, and our model can't capture that ceiling.

## EXERCISE 1: Exploring the scikit-learn Model

1. Make a prediction: what does the model predict for a district with median income of 5.0, house age of 20, average 5 rooms, average 1 bedroom, population of 1000, average occupancy of 3, latitude 34.0, and longitude -118.0? (Hint: you can pass a 2D array or DataFrame to `model_multi.predict()`)
2. Compute the **residuals** (actual - predicted) on the test set. What is the mean residual? What is the largest overprediction?
3. Create a bar chart of the feature coefficients.

In [ ]:
# 1. Predict for a specific district

# Write your code here



In [ ]:
#@title Click to reveal solution.

# Version 1: Raw np array
x_new = np.array([[5.0, 20, 5, 1, 1000, 3, 34.0, -118.0]])
predict_new = model_multi.predict(x_new)
print(f"Predicted median house value: ${predict_new[0] * 100_000:,.0f}")



In [ ]:

#@title Click to reveal solution.

# Version 2: Dataframe with named features
new_district = pd.DataFrame([{
    'MedInc': 5.0,
    'HouseAge': 20,
    'AveRooms': 5,
    'AveBedrms': 1,
    'Population': 1000,
    'AveOccup': 3,
    'Latitude': 34.0,
    'Longitude': -118.0
}])

predict_new = model_multi.predict(new_district)
print(f"Predicted median house value: ${predict_new[0] * 100_000:,.0f}")

In [ ]:
# 2. Compute residuals, mean residual, and largest overprediction

# Write your code here

In [ ]:
#@title Click to reveal solution.
residuals = y_test.values - y_pred_multi

print(f"Mean residual:         {residuals.mean():.4f}")
print(f"Largest overprediction: {residuals.min():.4f}")
print(f"  (actual was {residuals.min():.2f} lower than predicted)")

In [ ]:
# 3. Bar chart of feature coefficients

# Write your code here


In [ ]:
#@title Click to reveal solution.
coef_df = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': model_multi.coef_
}).sort_values('Coefficient')

plt.figure(figsize=(10, 5))
plt.barh(coef_df['Feature'], coef_df['Coefficient'])
plt.xlabel('Coefficient Value')
plt.title('Linear Regression Coefficients')
plt.axvline(x=0, color='black', linewidth=0.5)
plt.tight_layout()
plt.show()


---
# Section 2: Manual Least Squares with NumPy

Now let's peek under the hood. scikit-learn is solving the **Normal Equation**:

$$w^* = (A^T A)^{-1} A^T y$$

where **A** is the feature matrix with a column of 1s prepended (for the intercept).

Let's implement this ourselves with nothing but NumPy.

## Step 1: Build the Design Matrix

We need to add a column of ones to account for the intercept term. This turns our feature matrix **X** into the design matrix **A**.

In [ ]:
# Convert to NumPy arrays
X_train_np = X_train.values
y_train_np = y_train.values
X_test_np = X_test.values
y_test_np = y_test.values

print(f"X_train shape: {X_train_np.shape}")
print(f"y_train shape: {y_train_np.shape}")

In [ ]:
# Prepend a column of ones for the intercept
A_train = np.c_[np.ones(X_train_np.shape[0]), X_train_np]
A_test = np.c_[np.ones(X_test_np.shape[0]), X_test_np]

print(f"Design matrix A_train shape: {A_train.shape}")
print(f"\nFirst 3 rows of A_train (notice the leading 1s):")
print(A_train[:3, :4], "...")  # Show first 4 columns

## Step 2: Solve the Normal Equation

$$w^* = (A^T A)^{-1} A^T y$$

In NumPy, the `@` operator performs matrix multiplication, `.T` transposes, and `np.linalg.inv()` computes the inverse.

In [ ]:
# The Normal Equation in one line
w_star = np.linalg.inv(A_train.T @ A_train) @ A_train.T @ y_train_np

print(f"Weights shape: {w_star.shape}")
print(f"\nIntercept (w0): {w_star[0]:.4f}")
print(f"\nFeature weights:")
for name, w in zip(X.columns, w_star[1:]):
    print(f"  {name:>12s}: {w:.4f}")

## Step 3: Verify — Do We Match scikit-learn?

In [ ]:
# Compare our weights to sklearn's
print("Weight comparison (NumPy vs scikit-learn):")
print(f"{'':>12s}  {'NumPy':>10s}  {'sklearn':>10s}  {'Match?':>8s}")
print(f"{'Intercept':>12s}  {w_star[0]:>10.4f}  {model_multi.intercept_:>10.4f}  {'✓' if np.isclose(w_star[0], model_multi.intercept_) else '✗':>8s}")

for name, w_np, w_sk in zip(X.columns, w_star[1:], model_multi.coef_):
    match = '✓' if np.isclose(w_np, w_sk) else '✗'
    print(f"{name:>12s}  {w_np:>10.4f}  {w_sk:>10.4f}  {match:>8s}")

The weights are identical (within floating-point precision). scikit-learn is doing exactly this math under the hood.

In [ ]:
# Make predictions and evaluate
y_pred_numpy = A_test @ w_star

mse_numpy = np.mean((y_test_np - y_pred_numpy) ** 2)
r2_numpy = 1 - np.sum((y_test_np - y_pred_numpy) ** 2) / np.sum((y_test_np - np.mean(y_test_np)) ** 2)

print(f"NumPy   — MSE: {mse_numpy:.4f}, R²: {r2_numpy:.4f}")
print(f"sklearn — MSE: {mse_multi:.4f}, R²: {r2_multi:.4f}")

## A Note on `np.linalg.inv`

Computing the full matrix inverse is not the most numerically stable approach. In practice, `np.linalg.lstsq` uses a more robust method (SVD decomposition). Let's see:

In [ ]:
# More numerically stable approach
w_lstsq, residuals, rank, sv = np.linalg.lstsq(A_train, y_train_np, rcond=None)

print("Max difference between inv() and lstsq():")
print(f"  {np.max(np.abs(w_star - w_lstsq)):.2e}")
print("\nEffectively identical for this dataset.")

## EXERCISE 2: Simple Regression by Hand

For simple linear regression (one feature), the closed-form solution simplifies to:

- slope = covariance(x, y) / variance(x)
- intercept = mean(y) - slope * mean(x)

1. Using only `np.mean`, `np.var`, and `np.cov`, compute the slope and intercept for predicting house value from MedInc alone.
2. Compare your result to `model_simple.coef_` and `model_simple.intercept_` from Section 1.

Hint: `np.cov(a, b)` returns a 2×2 covariance matrix; the off-diagonal element `[0, 1]` is the covariance between a and b. Use `ddof=0` in `np.var` to match.

In [ ]:
# Compute slope and intercept for simple regression using the formulas
x_simple = X_train['MedInc'].values
y_simple = y_train.values

# Your code here:

In [ ]:
#@title Click to reveal solution.
cov_matrix = np.cov(x_simple, y_simple, ddof=0)
slope = cov_matrix[0, 1] / np.var(x_simple, ddof=0)
intercept = np.mean(y_simple) - slope * np.mean(x_simple)

print(f"Your slope:     {slope:.4f}  |  sklearn: {model_simple.coef_[0]:.4f}")
print(f"Your intercept: {intercept:.4f}  |  sklearn: {model_simple.intercept_:.4f}")

In [ ]:
# Compare to sklearn
# print(f"Your slope:     {slope:.4f}  |  sklearn: {model_simple.coef_[0]:.4f}")
# print(f"Your intercept: {intercept:.4f}  |  sklearn: {model_simple.intercept_:.4f}")


---
# Section 3: Gradient Descent with PyTorch

The Normal Equation gives us an exact answer, but it requires computing a matrix inverse — which becomes expensive with many features or samples. **Gradient descent** is an iterative alternative:

1. Start with random weights
2. Compute the loss (MSE)
3. Compute the gradient — which direction reduces the loss?
4. Take a small step in that direction
5. Repeat

This is how neural networks learn. We're introducing it here with linear regression so the concept is familiar when we get to deep learning.

In [ ]:
import torch

## Feature Scaling: Why It Matters for Gradient Descent

The Normal Equation doesn't care about the scale of features. Gradient descent does — a lot. If one feature ranges from 0–50 and another from 0–100,000, the gradients will be wildly different sizes and training becomes unstable.

We'll standardize the features to have mean 0 and standard deviation 1.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Before scaling — feature means:", X_train.mean().values.round(2))
print("After scaling  — feature means:", X_train_scaled.mean(axis=0).round(4))
print("After scaling  — feature stds: ", X_train_scaled.std(axis=0).round(4))

## Setting Up the Training Loop

In [ ]:
def run_gd(learning_rate, n_epochs, X_t, y_t, n_features):
    """Run gradient descent and return loss history."""
    torch.manual_seed(42)
    w = torch.randn(n_features, 1, requires_grad=True)
    b = torch.randn(1, requires_grad=True)
    loss_history = []

    for epoch in range(n_epochs):
        # Forward pass: compute predictions
        y_pred = X_t @ w + b

        # Compute MSE loss
        loss = torch.mean((y_pred - y_t) ** 2)
        loss_history.append(loss.item())

        # Backward pass: compute gradients
        loss.backward()

        # Update weights (gradient descent step)
        with torch.no_grad():
            w -= learning_rate * w.grad
            b -= learning_rate * b.grad

        # Zero out gradients for next iteration
        w.grad.zero_()
        b.grad.zero_()

        # Print progress every 200 epochs
        if (epoch + 1) % 200 == 0:
            print(f"Epoch {epoch+1:>4d}/{n_epochs} — Loss: {loss.item():.4f}")

    return w, b, loss_history

In [ ]:
# Convert to PyTorch tensors
X_t = torch.tensor(X_train_scaled, dtype=torch.float32)
y_t = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)

X_test_t = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_t = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

n_features = X_t.shape[1]
print(f"Training tensors: X={X_t.shape}, y={y_t.shape}")
print(f"Number of features: {n_features}")

In [ ]:
# Hyperparameters
learning_rate = 0.01
n_epochs = 1000

w, b, loss_history = run_gd(learning_rate, n_epochs, X_t, y_t, n_features)

print(f"\nFinal loss: {loss_history[-1]:.4f}")

## Visualizing the Training Process

One of the most important things to check: is the loss actually going down?

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(loss_history, linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.title('Training Loss Over Time')
plt.yscale('log')  # Log scale to see early and late progress
plt.grid(True, alpha=0.3)
plt.show()

The loss drops rapidly at first, then levels off as the weights approach the optimal values. This curve is typical of gradient descent — fast early progress followed by gradual refinement.

## Evaluating the Gradient Descent Model

In [ ]:
# Predict on test set
with torch.no_grad():
    y_pred_gd = (X_test_t @ w + b).numpy().flatten()

mse_gd = mean_squared_error(y_test, y_pred_gd)
r2_gd = r2_score(y_test, y_pred_gd)

print(f"Gradient Descent — MSE: {mse_gd:.4f}, R²: {r2_gd:.4f}")
print(f"scikit-learn    — MSE: {mse_multi:.4f}, R²: {r2_multi:.4f}")
print(f"NumPy (OLS)     — MSE: {mse_numpy:.4f}, R²: {r2_numpy:.4f}")

Gradient descent gets very close to the exact solution, but may not match it perfectly — it's an *approximation* that gets better with more epochs and careful tuning of the learning rate. For linear regression, OLS is better. But gradient descent scales to problems where OLS can't go: millions of parameters, nonlinear models, and neural networks.

## EXERCISE 3: Experiment with Gradient Descent

1. Copy the training loop from above and try `learning_rate = 0.1`. What happens? Then try `learning_rate = 0.001`. How does the loss curve change?
2. Try running for only 100 epochs vs 5000 epochs. How does the final MSE compare to scikit-learn?
3. (Bonus) Try removing the feature scaling step (use `X_train.values` instead of `X_train_scaled`). What goes wrong?

These experiments will give you intuition for how learning rate and scaling affect training.

In [ ]:
# 1. Compare learning rates

# Write your code here.


In [ ]:
#@title Click to reveal solution.

# 1. Compare learning rates

_w, _b, loss_slow = run_gd(0.001, 1000, X_t, y_t, n_features)
_w, _b, loss_medium = run_gd(0.01, 1000, X_t, y_t, n_features)
_w, _b, loss_fast = run_gd(0.1, 1000, X_t, y_t, n_features)

plt.figure(figsize=(10, 5))
plt.plot(loss_slow, label='lr=0.001 (too slow)')
plt.plot(loss_medium, label='lr=0.01')
plt.plot(loss_fast, label='lr=0.1 (faster)')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.title('Effect of Learning Rate')
plt.yscale('log')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"Final loss — lr=0.001: {loss_slow[-1]:.4f}")
print(f"Final loss — lr=0.01:  {loss_medium[-1]:.4f}")
print(f"Final loss — lr=0.1:   {loss_fast[-1]:.4f}")

In [ ]:
# 2. Compare different epoch counts

# Write your code here.

In [ ]:
#@title Click to reveal solution.

# 2. Compare different epoch counts
for epochs in [100, 1000, 5000]:
    _w, _b, history = run_gd(0.01, epochs, X_t, y_t, n_features)
    print(f"{epochs:>5d} epochs — Final MSE: {history[-1]:.4f}")

print(f"\nsklearn MSE (exact):   {mse_multi:.4f}")

In [ ]:
# 3. (Bonus) Remove feature scaling

In [ ]:
#@title Click to reveal solution.

# 3. (Bonus) Without feature scaling — loss explodes!
X_t_unscaled = torch.tensor(X_train.values, dtype=torch.float32)

try:
    _w, _b, loss_unscaled = run_gd(0.01, 100, X_t_unscaled, y_t, n_features)
    print(f"Final loss (unscaled): {loss_unscaled[-1]:.4f}")
    if np.isnan(loss_unscaled[-1]) or np.isinf(loss_unscaled[-1]):
        print("Loss exploded to NaN/Inf! Scaling is essential for gradient descent.")
except:
    print("Training failed — gradients exploded without scaling.")

# Even a tiny learning rate barely works
_w, _b, loss_unscaled_tiny = run_gd(0.0000001, 1000, X_t_unscaled, y_t, n_features)
print(f"\nWith lr=1e-7 (unscaled), final loss after 1000 epochs: {loss_unscaled_tiny[-1]:.4f}")
print(f"With lr=0.01 (scaled),   final loss after 1000 epochs:   {loss_medium[-1]:.4f}")
print("\nWithout scaling, you need an extremely small learning rate and many more epochs.")

---
# Section 4: Putting It All Together

Let's compare all three approaches side by side.

In [ ]:
results = pd.DataFrame({
    'Method': ['sklearn (1 feature)', 'sklearn (all features)', 'NumPy OLS', 'PyTorch GD'],
    'MSE': [mse_simple, mse_multi, mse_numpy, mse_gd],
    'R²': [r2_simple, r2_multi, r2_numpy, r2_gd]
})

print(results.to_string(index=False))

In [ ]:
# Visual comparison: predicted vs actual for all three multi-feature models
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

models_data = [
    ('scikit-learn', y_pred_multi, r2_multi),
    ('NumPy OLS', y_pred_numpy, r2_numpy),
    ('PyTorch GD', y_pred_gd, r2_gd)
]

for ax, (name, preds, r2) in zip(axes, models_data):
    ax.scatter(y_test, preds, alpha=0.15, s=8)
    ax.plot([0, 5], [0, 5], 'r--', linewidth=2)
    ax.set_xlabel('Actual')
    ax.set_ylabel('Predicted')
    ax.set_title(f'{name} (R² = {r2:.4f})')
    ax.set_xlim(0, 5.2)
    ax.set_ylim(0, 5.2)
    ax.set_aspect('equal')

plt.tight_layout()
plt.show()

## EXERCISE 4: Baseline Comparison

Remember from the lecture: the simplest "model" is to always predict the mean of the training target.

1. Compute the MSE and R² of a mean-only baseline (predict `y_train.mean()` for every test sample).
2. How much better is the full linear regression model compared to this baseline?
3. What R² value does the mean baseline give? Does that make sense mathematically?

In [ ]:
# Compute the mean baseline MSE and R²

# Write your code here.

In [ ]:
#@title Click to reveal solution.

# Mean baseline: predict y_train.mean() for every test sample
y_pred_baseline = np.full(len(y_test), y_train.mean())

mse_baseline = mean_squared_error(y_test, y_pred_baseline)
r2_baseline = r2_score(y_test, y_pred_baseline)

print(f"Mean baseline — MSE: {mse_baseline:.4f}, R²: {r2_baseline:.4f}")
print(f"Linear reg    — MSE: {mse_multi:.4f}, R²: {r2_multi:.4f}")
print(f"\nLinear regression reduces MSE by {(1 - mse_multi / mse_baseline) * 100:.1f}% over the baseline.")
print(f"\nR² of the mean baseline is ~0 by definition — it explains none of the variance.")
print(f"(The slight deviation from exactly 0 is because the test set mean differs slightly from the training set mean.)")

---
# Summary

You've now implemented linear regression three different ways:

- **scikit-learn**: The practical choice. Clean API, handles details for you. Use this in your projects.
- **NumPy (Normal Equation)**: Shows you the exact math. Gives the same answer as sklearn. Useful for understanding what's happening under the hood.
- **PyTorch (Gradient Descent)**: An iterative approximation. Slightly less precise for linear regression, but this exact approach scales to neural networks with millions of parameters.

Key takeaways:
- All three methods are solving the **same optimization problem**: minimize MSE
- More features generally help, but the improvement isn't always dramatic
- Gradient descent requires **feature scaling** and **hyperparameter tuning** (learning rate, epochs)
- Always compare against a simple **baseline** (like predicting the mean) to know if your model is actually useful